In [2]:
import pandas as pd

# do for 10 shows
from tqdm import tqdm
import time
import difflib
import json
from openai import OpenAI
import json
import os
import ast
import tiktoken
import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
from collections import defaultdict
import asyncio


In [87]:


with open("./categories_details.json", "r") as file:
    data = json.load(file)
all_categories = data["all_categories"]
category_rules = data["category_rules"]
avg_ad_price = data["avg_ad_price"]

In [4]:
pods = pd.read_csv("./1_pods_with_scrapped_data.csv")

episode_sponsors = pd.read_csv("./all_sponsors_of_shows.csv")


In [5]:
pods.head()

,id,name,itunesID,image,youtubeID,categories,summary,spotifyId,socialLinks,category,apple_reviews,followers_count,recent_posts,youtube_videos
0,20vc,The Twenty Minute VC (20VC),9.582305e+08,https://static.libsyn.com/p/assets/d/2/b/a/d2b...,UCf0PBRjhf0rF8fWBIxTuoWA,"[""business"",""society_&_culture"",""technology""]",The Twenty Minute VC brings insightful intervi...,3j2KMcZTtgTNBKwtZBMHvl,"{""facebook"":""https://www.facebook.com/The Twen...",business,"{'average_rating': 4.4, 'rating_count': 479, '...","{'instagram': 11470, 'youtube': 160000}",{'instagram': [{'caption': 'Cos all the best a...,"[{'age_restricted': False, 'airDate': '2025072..."
1,402p,4.0 To Pro,1.672464e+09,https://megaphone.imgix.net/podcasts/073cc458-...,UCzxISXhBIzA-2wsnC7z9_Aw,"[""sports"",""education"",""leisure""]",4.0 to Pro is a pocket-sized podcast dedicated...,5xQf95A7d9UESlpZi2YGBZ,"{""applePodcasts"":""https://podcasts.apple.com/u...",sports,"{'average_rating': 4.9, 'rating_count': 339, '...","{'instagram': 3454, 'youtube': 1060}",{'instagram': [{'caption': '@4.0_to_pro Custom...,"[{'age_restricted': False, 'airDate': '2025071..."
2,4thingswithamybrown,4 Things with Amy Brown,1.442391e+09,https://www.omnycontent.com/d/playlist/e73c998...,UC7qHbziuYTsDYp9OxYXOJOA,"[""health&fitness"",""society&_culture"",""education""]",Join Amy Brown as she discusses 4 essential pr...,7qMQUCmvUFlqZ0J68anixa,"{""applePodcasts"":""https://podcasts.apple.com/u...",health&fitness,"{'average_rating': 4.9, 'rating_count': 5232, ...",{'instagram': 623564},{'instagram': [{'caption': 'only the best for ...,NaN
3,99invisible,99% Invisible,3.947753e+08,https://storage.googleapis.com/core-dev-361305...,UCVMF2HD4ZgC0QHpU9Yq5Xrw,"[""arts"",""society_&_culture"",""education""]","Hosted by Roman Mars, 99% Invisible explores t...",2VRS1IJCTn2Nlkg33ZVfkM,"{""facebook"":""https://www.facebook.com/99percen...",society_&_culture,"{'average_rating': 4.8, 'rating_count': 25366,...","{'instagram': 87855, 'youtube': 11400}",{'instagram': [{'caption': '99% Invisible invi...,"[{'age_restricted': False, 'airDate': '2025071..."
4,Crimeatorium,Crimeatorium True Crime,1.724979e+09,https://media.redcircle.com/images/2024/12/10/...,UCM_TCZ3r8wQGfol-C6CSKBw,"[""true_crime"",""society_&_culture"",""education""]",Join Madeline O'Brien as she delves into lesse...,3DsvYLaAUl1NV0eUrr6uI1,"{""facebook"":""https://www.facebook.com/Crimeato...",true_crime,"{'average_rating': None, 'rating_count': None,...","{'instagram': 0, 'youtube': 21900}",{'instagram': []},[]


In [12]:
episode_sponsors.head()

,sponsorName,sponsorCode,startTime,endTime,domain,clientId,episodeId,episodeType,createdAt,airDate
0,Kalshi,NaN,4975.0,4984.0,kalshi.com,20vc,4lFDEWFzHVk,youtube,2025-07-26T17:16:36.460878,20250724
1,CRBN Pickleball,402p,94.0,162.0,crbnpickleball.com,402p,cLapzQiF0LM,youtube,2025-07-26T17:17:09.036052,20250719
2,Progressive,NaN,0.0,28.0,progressive.com,402p,hpas_GMyUQ0,youtube,2025-07-26T17:17:12.324929,20250719
3,Dropbox Dash,NaN,728.0,751.0,dropbox.com/dash,402p,hpas_GMyUQ0,youtube,2025-07-26T17:17:12.324952,20250719
4,Yobo,NaN,1328.0,1438.0,402p.com,402p,f6zt6XaXvy4,youtube,2025-07-26T17:17:16.461851,20250701


In [80]:

def analyze_sponsors(sponsors, videos):
    episode_durations = {v["id"]: v["length"] for v in videos}
    valid_episodes = {ep_id: duration for ep_id, duration in episode_durations.items() if duration > 1200}
    
    total_sponsor_ads = 0
    total_ad_time = 0.0
    domains = defaultdict(float)
    domain_counts = defaultdict(int)
    episodes_with_ads = set()
    episode_ad_time = defaultdict(float)  # Track ad time per episode
    episode_ad_counts = defaultdict(int)  # Track ad count per episode
    
    for s in sponsors:
        ep_id = s["episodeId"]
        # Only process sponsors for valid episodes (>20 minutes)
        if ep_id not in valid_episodes:
            continue
        
        sponsor_length = s["endTime"] - s["startTime"]
        domain = s["domain"]
        
        total_sponsor_ads += 1
        total_ad_time += sponsor_length
        domains[domain] += sponsor_length
        domain_counts[domain] += 1
        episodes_with_ads.add(ep_id)
        episode_ad_time[ep_id] += sponsor_length  # Add to episode's total ad time
        episode_ad_counts[ep_id] += 1  # Add to episode's ad count
    
    # Calculate average percentage of ads per episode
    episode_ad_percentages = []
    for ep_id, duration in valid_episodes.items():
        ad_time_in_episode = episode_ad_time.get(ep_id, 0)
        ad_percentage_in_episode = (ad_time_in_episode / duration) * 100
        episode_ad_percentages.append(ad_percentage_in_episode)
    
    avg_ad_percentage_per_episode = sum(episode_ad_percentages) / len(valid_episodes) if valid_episodes else 0
    
    # Calculate average episode ad count
    total_ads_across_episodes = sum(episode_ad_counts.values())
    avg_episode_ad_count = total_ads_across_episodes / len(valid_episodes) if valid_episodes else 0
    
    avg_sponsor_length = total_ad_time / total_sponsor_ads if total_sponsor_ads > 0 else 0
    unique_domains = len(domain_counts)
    avg_brand_repeat_rate = total_sponsor_ads / unique_domains if unique_domains > 0 else 0
    
    # Sort brands by frequency (most frequent first)
    brands_by_frequency = sorted(domain_counts.items(), key=lambda x: x[1], reverse=True)
    sorted_brands_by_frequency = [brand for brand, _ in brands_by_frequency]
    
    # Sort brands by total ad time (most advertised first)
    brands_by_ad_time = sorted(domains.items(), key=lambda x: x[1], reverse=True)
    sorted_brands_by_ad_time = [brand for brand, _ in brands_by_ad_time]

    one_pod_stats_df = pd.DataFrame()

    one_pod_stats = {
        "total_episodes": len(valid_episodes),
        "episodes_with_ads": len(episodes_with_ads),
        "percentage_episodes_with_ads": (len(episodes_with_ads) / len(valid_episodes) * 100) if valid_episodes else 0,
        "total_sponsor_ads": total_sponsor_ads,
        "total_ad_time": total_ad_time,
        "average_sponsor_length": avg_sponsor_length,
        "average_ad_percentage_per_episode": avg_ad_percentage_per_episode,
        "avg_episode_ad_count": avg_episode_ad_count,
        "sponsor_time_map": dict(domains),  # Domain -> total ad time
        "sponsor_frequency_map": dict(domain_counts),  # Domain -> total appearance count
        "average_brand_repeat_rate": avg_brand_repeat_rate,
        "sorted_brands_by_frequency": sorted_brands_by_frequency,  # [brand1, brand2, ...] sorted by frequency
        "sorted_brands_by_ad_time": sorted_brands_by_ad_time,  # [brand1, brand2, ...] sorted by ad time
    }

    one_pod_stats_df = pd.concat([one_pod_stats_df, pd.DataFrame([one_pod_stats])], ignore_index=True)
    
    return one_pod_stats_df

In [113]:
pods_sponsors_stats = pd.DataFrame()
    
    

In [114]:
# huberman_sponsors_pod = episode_sponsors.loc[episode_sponsors['clientId'] == 'hubermanlab']
# huberman_sponsors_list = huberman_sponsors_pod.to_dict('records')


# videos = pods.loc[pods['id'] == 'hubermanlab']["youtube_videos"]
# videos_array = ast.literal_eval(videos.iloc[0])  # Gets the first (and likely only) value

# no_sponsor_episodes = [video for video in videos_array if video.get("domain") != "nosponsors"]


# huberman_filtered_sponsors_list = [sponsor for sponsor in huberman_sponsors_list if sponsor.get("domain") != "nosponsors"]

# analyzed_report = analyze_sponsors(huberman_filtered_sponsors_list, videos_array)
# analyzed_report


for index, pod in tqdm(pods.iterrows(), total=len(pods)):
    pod_id = pod["id"]
    videos = pod["youtube_videos"]
    if pd.isna(videos):
        videos_array = []
    else:
        videos_array = ast.literal_eval(videos)

    pod_sponsers = episode_sponsors.loc[episode_sponsors['clientId'] == pod_id]
    pod_sponsers = pod_sponsers.to_dict('records')
    df = analyze_sponsors(filtered_sponsors_list, videos_array)
    df["clientId"] = pod_id
    pods_sponsors_stats = pd.concat([pods_sponsors_stats, df], ignore_index=True)
    # df = await generate_and_store_sponsor_for_show(pod, index)
    # all_sponsors_df = pd.concat([all_sponsors_df, df], ignore_index=True)

  0%|          | 0/275 [00:00<?, ?it/s]


NameError: name 'analyze_sponsors' is not defined

In [115]:
pods_sponsors_stats

""


In [ ]:
pods_stats.to_csv("pods_sponsora_stats.csv", index=False)


In [76]:

def match_category(input_category: str) -> str:
    input_category = input_category.lower().strip()
    for keyword, mapped_category in category_rules.items():
        if keyword in input_category:
            return mapped_category
    closest_match = difflib.get_close_matches(input_category, all_categories, n=1, cutoff=0.4)
    return closest_match[0] if closest_match else ""

In [110]:
def get_show_size(impressions):
    if impressions < 10000:
        return "micro"
    elif impressions < 100000:
        return "mid_tier"
    else:
        return "large"


In [111]:
def get_estimated_impressions(pod):
    apple_reviews = pod["apple_reviews"]
    apple_reviews = ast.literal_eval(apple_reviews)

    total_ratings = apple_reviews["rating_count"]
    if total_ratings is not None:
        min_impressions = total_ratings * 15
        avg_impressions = total_ratings * 20
        max_impressions = total_ratings * 25
        return min_impressions, avg_impressions, max_impressions
    else:
        return 0, 0, 0


def get_social_media_count(pod):
    followers_count = pod["followers_count"]

    followers_count = ast.literal_eval(followers_count)
    youtube_subscribers = followers_count.get("youtube", 0)
    instagram_followers = followers_count.get("instagram", 0)

    
    return youtube_subscribers, instagram_followers
    
    

def youtube_per_episode_avg_views(pod):
    youtube_videos = pod["youtube_videos"]
    if youtube_videos is  None or pd.isna(youtube_videos):
        youtube_videos  = "[]"
    youtube_videos = ast.literal_eval(youtube_videos)

    total_views = 0
    for video in youtube_videos:
        stats = video["stats"]
        if stats is not None:
            total_views += stats["views"]
    if len(youtube_videos) > 0:
        return total_views / len(youtube_videos)
    else:
        return 0


def estimated_price_per_ad(pod):
    category = pod["category"]
    category = match_category(category)
    if category == "":
        category_ads = avg_ad_price["unknown"]
    else:
        category_ads = avg_ad_price[category]

    _, avg_impressions, _ = get_estimated_impressions(pod)
    show_size = get_show_size(avg_impressions)
    ad_prices = category_ads[show_size]
    return ad_prices

In [135]:
pods_data_stats = pd.DataFrame()

In [136]:
for index, pod in tqdm(pods.iterrows(), total=len(pods)):
    min_impressions, avg_impressions, max_impressions = get_estimated_impressions(pod)
    youtube_subscribers, instagram_followers  = get_social_media_count(pod)
    episode_avg_views = youtube_per_episode_avg_views(pod)
    estimated_ad_price = estimated_price_per_ad(pod)


    one_pod_stats_df = pd.DataFrame()

    one_pod_stats = {
        "clientId": pod["id"],
        "min_impressions": min_impressions,
        "max_impressions": max_impressions,
        "youtube_subscribers": youtube_subscribers,
        "instagram_followers": instagram_followers,
        "episode_avg_views": episode_avg_views,
        "estimated_ad_price": estimated_ad_price
    }
    one_pod_stats_df = pd.concat([one_pod_stats_df, pd.DataFrame([one_pod_stats])], ignore_index=True)
    
    print("pod", pod["id"])
    pods_data_stats = pd.concat([pods_data_stats, one_pod_stats_df], ignore_index=True)
    

100%|██████████| 275/275 [00:00<00:00, 1992.64it/s]

pod 20vc
pod 402p
pod 4thingswithamybrown
pod 99invisible
pod Crimeatorium
pod Datingintelligence
pod MidnightBurger
pod SPNthenandnow
pod Travel-in-10
pod adatewithdateline
pod afterhoursentrepreneur
pod alignpodcast
pod allin
pod almostadulting
pod almostfamous
pod andthatswhywedrink
pod anotherkingdom
pod anxietycoachespod
pod arenafootballleague
pod armchairexpert
pod arrowheadpride
pod asianbossgirl
pod askacyclingcoach
pod athleticomince
pod badfriends
pod badtv
pod beargrease
pod beermethatmovie
pod behindthevelvetrope
pod betterthanhappy
pod bewellby
pod bigdeal
pod bourbonpursuit
pod businesswars
pod cabinetofcuriosities
pod caliphate
pod callherdaddy
pod carefreeblackgirl
pod caseyadams
pod chadprather
pod chapotraphouse
pod chrismoylesshow
pod clearedhot
pod cocktales
pod codestory
pod codeswitch
pod congratulations
pod conversationswithcam
pod crimeinsports
pod criminology
pod curious
pod dailymeditationpodcast
pod dailyradioprogram
pod daminternetyouscary
pod darknetdiarie

In [137]:
pods_data_stats

,clientId,min_impressions,max_impressions,youtube_subscribers,instagram_followers,episode_avg_views,estimated_ad_price
0,20vc,7185,11975,160000,11470,14536.0,"{'30s': '14,20', '60s': '18,25'}"
1,402p,5085,8475,1060,3454,179.2,"{'30s': '10,15', '60s': '14,20'}"
2,4thingswithamybrown,78480,130800,0,623564,0.0,"{'30s': '24,30', '60s': '30,38'}"
3,99invisible,380490,634150,11400,87855,1919.6,"{'30s': '20,26', '60s': '26,33'}"
4,Crimeatorium,0,0,21900,0,0.0,"{'30s': '11,16', '60s': '15,21'}"
...,...,...,...,...,...,...,...
270,wsjtechnews,23520,39200,142000,5398554,569.4,"{'30s': '22,28', '60s': '28,35'}"
271,yoretown,30,50,238,0,42.4,"{'30s': '11,16', '60s': '15,21'}"
272,yourfavoritethingwithwellsandbrandi,82005,136675,1630,13665,837.4,"{'30s': '19,25', '60s': '25,32'}"
273,yourmomdad,348045,580075,37500,50784,7583.4,"{'30s': '19,25', '60s': '25,32'}"


In [138]:
pods_data_stats.to_csv("pods_stats.csv", index=False)